[View in Colaboratory](https://colab.research.google.com/github/tifa64/Modulation-Classification/blob/master/Radio_Signal_Processing.ipynb)

In [2]:
!pip install tensorflow
!pip install keras
!pip install pandas
!pip install -U -q PyDrive

In [3]:
!rm -rf datalab
!rm 2016.04C.multisnr.pkl.bz2 
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!ls
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls

rm: cannot remove '2016.04C.multisnr.pkl.bz2': No such file or directory
--2018-05-08 19:51:35--  http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
Connecting to 104.197.136.14:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111458161 (106M) [application/x-bzip2]
Saving to: ‘2016.04C.multisnr.pkl.bz2’

2016.04C.multisnr.p 100%[===================>] 106.29M  63.0MB/s    in 1.7s    

2018-05-08 19:51:36 (63.0 MB/s) - ‘2016.04C.multisnr.pkl.bz2’ saved [111458161/111458161]

2016.04C.multisnr.pkl.bz2
2016.04C.multisnr.pkl


In [0]:
import pickle
import numpy as np
import os
import pandas as pd
from scipy import integrate
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def f(x,c):
  return x*c

In [0]:
with open('2016.04C.multisnr.pkl', 'rb') as f:
    Xd = pickle.load(f,encoding='bytes')

mods,snrs = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [0,1])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)


# Partition the data

In [6]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
print(n_train)
train_idx = np.random.choice(range(0, n_examples), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

81030.0


# Adding diffrentiation feature

In [7]:
X_train_def = np.empty([81030, 4, 128])
for i,sample in  enumerate(X_train):
    for vector in sample:
        grad = np.gradient(vector)
        sample = np.vstack((sample,grad))
    X_train_def[i] = sample   
print(X_train_def[0][2][0])    

0.5915333032608032


In [0]:
X_train_def2 = np.empty([81030, 2, 128])
container = np.empty([2, 128])
for i,sample in  enumerate(X_train):
    for j, vector in enumerate(sample):
        grad = np.gradient(vector)
        container[j] = grad
    X_train_def2[i] = container
    container = np.empty([2, 128])


In [9]:
def f(x,c):
  return x*c

X_train_int2 = np.empty([81030, 2, 128])
print(X_train[0][0][0])
for i,sample in enumerate(X_train):
    for j, vector in enumerate(sample):
        for k, element in enumerate(vector):
            X_train_int2[i][j][k] = integrate.quad(f, 0, 0.128, element)[0]
       

-1.6880678


In [26]:
def indices_merged_arr(arr):
    n = arr.ndim
    grid = np.ogrid[tuple(map(slice, arr.shape))]
    out = np.empty(arr.shape + (n+1,), dtype=arr.dtype)
    for i in range(n):
        out[...,i+1] = grid[i]
    out[...,0] = arr
    out.shape = (-1,n+1)
    return out
  

print( os.getcwd() )
print(X_train_int2.shape)
# stacked = pd.Panel(X_train_int2.swapaxes(1,2)).to_frame().stack().reset_index()
# stacked.columns = ['x', 'y', 'z', 'value']
# # save to disk
# stacked.to_csv("matrix2.csv", header=None)
df = pd.DataFrame(indices_merged_arr(X_train_int2), columns=list('Axyz'))
print(df.get_values)
df.to_csv("matrix3.csv")

/content
(81030, 2, 128)
<bound method NDFrame.get_values of                  A        x    y      z
0        -0.013829      0.0  0.0    0.0
1        -0.008983      0.0  0.0    1.0
2        -0.011593      0.0  0.0    2.0
3        -0.011579      0.0  0.0    3.0
4        -0.016849      0.0  0.0    4.0
5        -0.016577      0.0  0.0    5.0
6        -0.012991      0.0  0.0    6.0
7        -0.013456      0.0  0.0    7.0
8        -0.012640      0.0  0.0    8.0
9        -0.011822      0.0  0.0    9.0
10       -0.016012      0.0  0.0   10.0
11       -0.012626      0.0  0.0   11.0
12       -0.013330      0.0  0.0   12.0
13       -0.016067      0.0  0.0   13.0
14       -0.015409      0.0  0.0   14.0
15       -0.012156      0.0  0.0   15.0
16       -0.012966      0.0  0.0   16.0
17       -0.013079      0.0  0.0   17.0
18       -0.013808      0.0  0.0   18.0
19       -0.014094      0.0  0.0   19.0
20       -0.013424      0.0  0.0   20.0
21       -0.013021      0.0  0.0   21.0
22       -0.012249 

In [0]:
# Code to read csv file into colaboratory:


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1Q_tifiTlVE-bJgXE3TXtspdzRdTF56_m'}) # replace the id with id of file you want to access
downloaded.GetContentFile('matrix3.csv')  

#3. Read file as panda dataframe
import pandas as pd
xyz = pd.read_csv('matrix3.csv') 



In [34]:
print(xyz.get_values)
X_int = np.array(xyz['A'])
#X_int = np.insert(X_int, 0, (xyz.keys()[4]))
X_int = X_int.reshape((81030, 2, 128)) 



<bound method NDFrame.get_values of           Unnamed: 0         A        x    y      z
0                  0 -0.013829      0.0  0.0    0.0
1                  1 -0.008983      0.0  0.0    1.0
2                  2 -0.011593      0.0  0.0    2.0
3                  3 -0.011579      0.0  0.0    3.0
4                  4 -0.016849      0.0  0.0    4.0
5                  5 -0.016577      0.0  0.0    5.0
6                  6 -0.012991      0.0  0.0    6.0
7                  7 -0.013456      0.0  0.0    7.0
8                  8 -0.012640      0.0  0.0    8.0
9                  9 -0.011822      0.0  0.0    9.0
10                10 -0.016012      0.0  0.0   10.0
11                11 -0.012626      0.0  0.0   11.0
12                12 -0.013330      0.0  0.0   12.0
13                13 -0.016067      0.0  0.0   13.0
14                14 -0.015409      0.0  0.0   14.0
15                15 -0.012156      0.0  0.0   15.0
16                16 -0.012966      0.0  0.0   16.0
17                17 -0.0130